In [55]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime
import ast
from itertools import chain

In [56]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BAQ'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 0

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

### Loading COSTUMERS INFO

In [57]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [58]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [59]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [60]:
data_ventas = data_ventas.groupby(by=['region_code','customer_id']).agg({"gmv_usd":np.sum, "order_id":pd.Series.nunique}).reset_index()
data_ventas["AOV"] = data_ventas.gmv_usd/data_ventas.order_id

In [61]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (
SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('day', date(getdate()) - interval '30 day')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}'
),

view_sku AS (
SELECT DISTINCT
  --ds.identifier_value,
  s.site_identifier_value,
  --ea.ctsessionid,
  ea.profile_identity,
  ea.frproductid,
  bs2.name,
  bcat2.name AS category,
  bcat.name AS subcat,
  --ea.frproductname,
  --ea.frproductinstock,
  DATE_TRUNC('day', ea.event_date) AS event_date
  --ea.eventname,
  --ea.frrecommendationtype,
  --ea.frfromproductid
  --ea.frscreen--,
  --ea.frsubmitresult--,--para el cart add
  --info_subs.num_substitutes
FROM dpr_clevertap.events_analytics                                     ea
INNER JOIN dpr_clevertap.user_profile                                   p       ON ea.profile_identity = p.profile_identity AND ea.profile_identity NOT ILIKE '%,%'
INNER JOIN dpr_shared.dim_site                                          ds      ON ds.identifier_value = p.registeredcity
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"          bp      ON bp.product_id::varchar = ea.frproductid::varchar
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"             s       ON s.site_id = bp.catalog_disc AND ds.identifier_value = s.site_identifier_value
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"    bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat2   ON bcx.category_id = bcat2.category_id

WHERE
  ea.eventname = 'FRF Product View'
  AND ea.event_date > current_date - 30
  AND ds.identifier_value = '{city}'
 ),
 
almost AS (

SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    ful.submit_date,
    --bs2.name,
    bs.addl_product_id as padre_sku_id
    --bcat2.name AS category,
    --bcat.name AS subcat,
    -- CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    -- ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    -- CASE WHEN s.site_identifier_value = 'CMX' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'GDL' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'PBC' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'MTY' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'SPO' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'BHZ' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'CWB' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'VCP' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    -- ELSE (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/3776) END AS discount_usd,
    -- SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND bo.customer_id IN (SELECT DISTINCT profile_identity FROM view_sku)
)

SELECT DISTINCT
    vs.site_identifier_value AS city,
    vs.profile_identity AS customer_id,
    vs.frproductid AS card_id,
    vs.name AS product,
    vs.category AS cat,
    vs.subcat
FROM view_sku vs
LEFT JOIN almost a ON a.customer_id = vs.profile_identity 
 AND a.city = vs.site_identifier_value 
 AND a.padre_sku_id = vs.frproductid 
 AND a.submit_date BETWEEN vs.event_date AND DATEADD(day, 1, vs.event_date)
-- AND (a.submit_date = vs.event_date OR a.submit_date = DATEADD(day, 5, vs.event_date))

WHERE a.customer_id IS NULL
""".format(city = city_code)

df_view = read_connection_data_warehouse.run_read_prod_query(query)
df_view[["customer_id", "card_id"]] = df_view[["customer_id", "card_id"]].astype(int)

In [62]:
from sklearn.model_selection import train_test_split 

def create_groups(offer):
    
    customers = offer.customer_id.unique()
    
    customer_control, customer_test = train_test_split(customers, test_size = 0.80)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['AOV'].mean()/offer[offer.customer_id.isin(customer_test)]['AOV'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.80)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [63]:
def get_info(df, sku):
    control, test = create_groups(data_ventas.loc[data_ventas.customer_id.isin(df.customer_id.unique())])
    name = df["product"].unique()[0].replace(",", ".")
    
    print(name+": \nCONTROL: ",len(control)," PILOTO: ",len(test))
    print("MARGEN: ", df_margin.loc[(df_margin.id_tarjeta == sku) & (df_margin.day == df_margin.day.max()),"margin"].values[0].round(3)*100)
    
    return control, test, name

# Loading data

In [64]:
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging
import datetime
import math

In [65]:
current_week_number = datetime.datetime.now().isocalendar()[1]
print(current_week_number)

current_week_number2 = current_week_number #+ 1
print(current_week_number2) 

23
23


In [66]:
paramss_csv=GoogleSheet("1BrmB2Wv1bkoIdAA8DrJgZRxqapo-GY47VC879p_zrfc")
city_paramss = paramss_csv.get_as_dataframe('TOTAL')
city_paramss = city_paramss.loc[city_paramss.City == city_code,['City','Info','Subcat',current_week_number2]]

In [67]:
city_paramss

,City,Info,Subcat,23
24,BAQ,products,"Café, Chocolate e Infusiones",NaN
25,BAQ,dct,"Café, Chocolate e Infusiones",NaN
26,BAQ,kind,"Café, Chocolate e Infusiones",NaN
27,BAQ,relation,"Café, Chocolate e Infusiones",NaN
28,BAQ,products,Aceites & Grasas,"['259559','180812','200951']"
29,BAQ,dct,Aceites & Grasas,"[5,3,5]"
30,BAQ,kind,Aceites & Grasas,SD
31,BAQ,relation,Aceites & Grasas,OK
32,BAQ,products,Pollo Congelado,"['156059','117079','117309']"
33,BAQ,dct,Pollo Congelado,"[5,5,5]"


In [68]:
city_paramss = city_paramss.dropna()

In [69]:
skus = []
discount = []
kind = []
subcat = []
for x in city_paramss.values:
    if x[1] == 'products':
        skus.append(x[3])
    elif x[1] == 'dct':
        discount.append(x[3])
    elif x[1] == 'relation':
        subcat.append(x[3])
    elif x[1] == 'kind':
        kind.append(x[3])

In [70]:
"""Here we should complete our data for each SH, important if we want to submit a subcategory SH offer, in SKUS we should put the skus into a list as this [[1,2,3]]"""
#offers_name = [("BET_"+x).replace(",",".") for x in city_paramss.Subcat.unique()]
card_id = [eval(x) for x in skus]
discount = discount
max_uses = [30]
max_orders = [2]
#////////////
subcats = subcat
users = []

In [71]:
count=0
for i in range(0,len(subcat)):
    if kind[i] == 'SD':
        #offers_name.pop(i-count)
        card_id.pop(i-count)
        discount.pop(i-count)
        subcats.pop(i-count)
        count+=1
        
card_id = [int(num) for sublist in card_id for num in sublist]

nested_list = [ast.literal_eval(string) for string in discount]
discount = list(chain.from_iterable(nested_list))

In [72]:
offer_name = []
global_segment_list = [] #//for procesing
for sku in card_id:
    print(sku)
    if len(df_view.loc[df_view.card_id == sku]) != 0:
        print(len(df_view.loc[df_view.card_id == sku]))
        control,test,name = get_info(df_view.loc[df_view.card_id == sku], sku)

        offer_name.append(("SH_VIEW_BET_"+todays_date+"_"+city_code+"_"+name.replace(",",".")).strip())
        d = {'name':("SH_VIEW_BET_"+todays_date+"_"+city_code+"_"+name.replace(",",".")).strip(), 'customersIds': list(test)}
        e = {'name':("SH_VIEW_BET_"+todays_date+"_"+city_code+"_"+name.replace(",",".")).strip()+"_CONTROL", 'customersIds': list(control)}
        
        global_segment_list.append(d)
        global_segment_list.append(e)
    else:
        print("0 BUSQUEDAS")

156059
1587
Pechuga con Hueso y Piel  Campollo x 2.7kg - 3.5kg: 
CONTROL:  180  PILOTO:  720
MARGEN:  13.700000000000001
117079
466
Pechuga x 1kg  Mercapollo: 
CONTROL:  49  PILOTO:  200
MARGEN:  12.3
117309
498
Muslo con Contra Muslo Mercapollo x 500gr : 
CONTROL:  48  PILOTO:  192
MARGEN:  12.0
224616
90
Servilleta Popular x 600 und: 
CONTROL:  15  PILOTO:  63
MARGEN:  19.8
352210
21
Toalla en Z natural DH Nube x 150 h: 
CONTROL:  3  PILOTO:  13
MARGEN:  18.099999999999998
224610
41
Papel Higienico rendidor Blanko x 30 m: 
CONTROL:  4  PILOTO:  18
MARGEN:  22.5


In [73]:
csv_file = {} 
csv_file['offer_name'] = offer_name
csv_file['offer_description'] = None
csv_file['discount'] = discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+7))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = max_uses*len(offer_name)
csv_file['max_uses_per_customer'] = max_orders*len(offer_name)
csv_file['customer_segment_id'] = [1]*len(offer_name) #[None]
csv_file['sku_id'] = [frida_products.loc[frida_products.id_tarjeta == sku,"skus"].values[0] for sku in card_id]

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [74]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y...,None,5,true,05-06-2023,12-06-2023,30,2,1,200610#200608
1,SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg Merc...,None,5,true,05-06-2023,12-06-2023,30,2,1,145410
2,SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Mu...,None,5,true,05-06-2023,12-06-2023,30,2,1,145690
3,SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular ...,None,8,true,05-06-2023,12-06-2023,30,2,1,282562
4,SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural...,None,8,true,05-06-2023,12-06-2023,30,2,1,471944
5,SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico ren...,None,8,true,05-06-2023,12-06-2023,30,2,1,282543


In [75]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [76]:
response.text

'Customer segments were created: [171963-SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y Piel  Campollo x 2.7kg - 3.5kg, 171964-SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y Piel  Campollo x 2.7kg - 3.5kg_CONTROL, 171965-SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg  Mercapollo, 171966-SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg  Mercapollo_CONTROL, 171967-SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Muslo Mercapollo x 500gr, 171968-SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Muslo Mercapollo x 500gr_CONTROL, 171969-SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular x 600 und, 171970-SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular x 600 und_CONTROL, 171971-SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural DH Nube x 150 h, 171972-SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural DH Nube x 150 h_CONTROL, 171973-SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico rendidor Blanko x 30 m, 171974-SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico rendidor Blanko x 30 m_CONTROL]'

In [77]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [78]:
sgt

[(171963,
  'SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y Piel  Campollo x 2.7kg - 3.5kg'),
 (171964,
  'SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y Piel  Campollo x 2.7kg - 3.5kg_CONTROL'),
 (171965, 'SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg  Mercapollo'),
 (171966, 'SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg  Mercapollo_CONTROL'),
 (171967,
  'SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Muslo Mercapollo x 500gr'),
 (171968,
  'SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Muslo Mercapollo x 500gr_CONTROL'),
 (171969, 'SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular x 600 und'),
 (171970, 'SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular x 600 und_CONTROL'),
 (171971, 'SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural DH Nube x 150 h'),
 (171972,
  'SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural DH Nube x 150 h_CONTROL'),
 (171973, 'SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico rendidor Blanko x 30 m'),
 (171974,
  'SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico rendidor Blanko x 30 m_CONTROL')]

In [79]:
for i in global_offer_csv.offer_name:
    for j,z in sgt:
        if z==i:
            global_offer_csv.loc[global_offer_csv.offer_name == i, ('customer_segment_id')] = j
            break

In [80]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_VIEW_BET_2023-06-05_BAQ_Pechuga con Hueso y...,None,5,true,05-06-2023,12-06-2023,30,2,171963,200610#200608
1,SH_VIEW_BET_2023-06-05_BAQ_Pechuga x 1kg Merc...,None,5,true,05-06-2023,12-06-2023,30,2,171965,145410
2,SH_VIEW_BET_2023-06-05_BAQ_Muslo con Contra Mu...,None,5,true,05-06-2023,12-06-2023,30,2,171967,145690
3,SH_VIEW_BET_2023-06-05_BAQ_Servilleta Popular ...,None,8,true,05-06-2023,12-06-2023,30,2,171969,282562
4,SH_VIEW_BET_2023-06-05_BAQ_Toalla en Z natural...,None,8,true,05-06-2023,12-06-2023,30,2,171971,471944
5,SH_VIEW_BET_2023-06-05_BAQ_Papel Higienico ren...,None,8,true,05-06-2023,12-06-2023,30,2,171973,282543


In [81]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_SH_PLUS_{city_code}_OFFER_csv.csv', index = False)